CIFAR-10

https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html#sphx-glr-beginner-blitz-cifar10-tutorial-py

# Import modules

In [23]:
import copy
from datetime import datetime
import gc
import logging
from logging import getLogger
from logging import StreamHandler
from logging import INFO, WARNING
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
from tensorboardX import SummaryWriter
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import warnings
warnings.filterwarnings("ignore")

logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(INFO)
logger.setLevel(INFO)
logger.addHandler(handler)

%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


# Global vars

In [26]:
TRAIN = "train"
VAL = "val"
TEST = "test"
PHASES = [TRAIN, VAL]

SAVE_PATH = os.path.join(os.getcwd(), "models")

# Utils

In [ ]:
def display_formatted_time(elapsed_time, msg=""):
    minutes, seconds = map(int, divmod(elapsed_time, 60))
    print("Elapsed time - {0}: {1}min {2}s".format(msg, minutes, seconds))

In [ ]:
def save_model(model_name=""):
    model_path = os.path.join(SAVE_PATH, model_name)
    torch.save(model.state_dict(), model_path)

def load_model(model_name):
    model = Net().to(device)
    model_path = os.path.join(SAVE_PATH, model_name)
    model.load_state_dict(torch.load(model_path))
    return model

In [35]:
def generate_log_path():
    # return os.path.join("logs", "train_{}".format(datetime.utcnow().strftime("%Y%m%d%H%M%S")))
    return os.path.join("logs", "train")

# Settings

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ", device)

torch.manual_seed(1)

test_batch_size = 256

# default
params = {
    "batch_size": 512,
    "epochs": 10,
    "lr": 0.001,
    "momentum": 0.9,
}

# Load image

In [ ]:
def init_datasets(data_transforms):
    """画像前処理変更したら呼ぶ"""
    image_datasets = {phase: torchvision.datasets.CIFAR10(root="../../data",
                                                                                                       train=phase is "train",
                                                                                                       download=True,
                                                                                                       transform=data_transforms[phase])
                                      for phase in PHASES}
    dataset_sizes = {phase: len(image_datasets[phase]) for phase in PHASES}
    return image_datasets, dataset_sizes

def init_dataloaders(batch_size, image_datasets):
    """バッチサイズ変更したら呼ぶ"""
    return {phase: torch.utils.data.DataLoader(image_datasets[phase],
                                                                               batch_size=batch_size,
                                                                               shuffle=True,
                                                                               num_workers=4)
                 for phase in ["train", "val"]}

In [ ]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

classes = (
    "plane",
    "car",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck"
)

In [ ]:
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

dataiter = iter(dataloaders[TRAIN])
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))
print(" ".join("%5s" % classes[labels[j]] for j in range(params["batch_size"])))

# Define a Convolution Neural Network

In [ ]:
# from net import Net

In [9]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net,  self).__init__()
        self.conv1 = nn.Conv2d(3,  16,  5)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(2,  2)
        self.conv2 = nn.Conv2d(16,  16,  5)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5,  120)
        self.fc2 = nn.Linear(120,  84)
        self.fc3 = nn.Linear(84,  10)

    def forward(self,  x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))  # 32x32x3 ->  28x28x6 ->14x14x6
        x = self.pool(F.relu(self.bn2(self.conv2(x))))  #  ->10x10x16 -> 5x5x16
        x = x.view(-1,  16 * 5 * 5)  # -> 400
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

ResNet

# Train the network

In [33]:
def train(model, criterion, optimizer, scheduler, params):
    since = time.time()
    
    # Tensorboard
    log_path = generate_log_path()
    writer = SummaryWriter(log_path)
    
    epochs = params["epochs"]

    best_model_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(epochs):
        epoch_loss = dict()
        epoch_acc = dict()
        
        for phase in PHASES:
            if phase == TRAIN:
                is_train = True
                scheduler.step()
                model.train()
            else:
                is_train = False
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            
            for batch_idx, (inputs, labels) in enumerate(dataloaders[phase], 0):
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(is_train):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if is_train:
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss[phase] = running_loss / dataset_sizes[phase]
            epoch_acc[phase] = running_corrects.double() / dataset_sizes[phase]

            if phase == VAL and epoch_acc[phase] > best_acc:
                best_acc = epoch_acc[phase]
                best_model_weights = copy.deepcopy(model.state_dict())

        print("Epoch {}/{}\tTrain Loss: {:.4f} Acc: {:.4f}\tVal Loss: {:.4f} Acc: {:.4f}".format(
            epoch,
            epochs - 1,
            epoch_loss[TRAIN],
            epoch_acc[TRAIN],
            epoch_loss[VAL],
            epoch_acc[VAL]
        ))
        writer.add_scalars(
            "losses",
            {
                "train_loss": epoch_loss[TRAIN],
                "val_loss": epoch_loss[VAL],
            },
            epoch
        )
        writer.add_scalars(
            "acc",
            {
                "train_acc": epoch_acc[TRAIN],
                "val_acc": epoch_acc[VAL]
            },
            epoch
        )

    print()
    print("Best val Acc: {:4f}".format(best_acc))
    
    writer.close()
    model.load_state_dict(best_model_weights)
    
    print()
    display_formatted_time(time.time() - since)
    return model

In [31]:
def calc_acc(model):
    """ラベルごとの精度を算出"""
    class_correct = [0. for i in range(len(classes))]
    class_total = [0. for i in range(len(classes))]

    with torch.no_grad():
        for inputs, labels in dataloaders[VAL]:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print("Accuracy of\t%5s:\t%2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))

## Train and Eval

In [ ]:
params = {
    "batch_size": 1024,
    "epochs": 50,
    "lr": 0.001,
    "momentum": 0.9,
}

image_datasets, dataset_sizes = init_datasets(data_transforms=data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets=image_datasets)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(),
#                                          lr=params["lr"],
#                                          momentum=params["momentum"])
optimizer = optim.Adam(model.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                                              T_max=10,  # Maximum number of iterations
                                                                              eta_min=0,  # 最小学習率
                                                                              last_epoch=-1)  # The index of last epoch

model = train(model, criterion, optimizer, scheduler, params)

Files already downloaded and verified
Files already downloaded and verified
Epoch 0/49	Train Loss: 1.8440 Acc: 0.3370	Val Loss: 1.6088 Acc: 0.4023
Epoch 1/49	Train Loss: 1.4035 Acc: 0.4877	Val Loss: 1.3513 Acc: 0.5073
Epoch 2/49	Train Loss: 1.2661 Acc: 0.5415	Val Loss: 1.2507 Acc: 0.5483
Epoch 3/49	Train Loss: 1.1900 Acc: 0.5730	Val Loss: 1.2773 Acc: 0.5451
Epoch 4/49	Train Loss: 1.1387 Acc: 0.5917	Val Loss: 1.1703 Acc: 0.5860
Epoch 5/49	Train Loss: 1.1071 Acc: 0.6066	Val Loss: 1.1114 Acc: 0.6057
Epoch 6/49	Train Loss: 1.0812 Acc: 0.6154	Val Loss: 1.1397 Acc: 0.6019
Epoch 7/49	Train Loss: 1.0625 Acc: 0.6253	Val Loss: 1.0928 Acc: 0.6152
Epoch 8/49	Train Loss: 1.0520 Acc: 0.6277	Val Loss: 1.0763 Acc: 0.6174
Epoch 9/49	Train Loss: 1.0435 Acc: 0.6314	Val Loss: 1.0721 Acc: 0.6220
Epoch 10/49	Train Loss: 1.0440 Acc: 0.6289	Val Loss: 1.0719 Acc: 0.6223
Epoch 11/49	Train Loss: 1.0431 Acc: 0.6307	Val Loss: 1.0711 Acc: 0.6226
Epoch 12/49	Train Loss: 1.0435 Acc: 0.6297	Val Loss: 1.0677 Acc: 0.622

In [15]:
calc_acc(model)

Accuracy of	plane:	67 %
Accuracy of	  car:	77 %
Accuracy of	 bird:	49 %
Accuracy of	  cat:	37 %
Accuracy of	 deer:	54 %
Accuracy of	  dog:	56 %
Accuracy of	 frog:	76 %
Accuracy of	horse:	69 %
Accuracy of	 ship:	75 %
Accuracy of	truck:	75 %


## Save model

In [ ]:
save_model("original_Adam_epoch50")

## Load model

In [10]:
model = load_model("original_SGD_epoch50")

# Learning rate annealing

Cosine Annealing

In [ ]:
params = {
    "batch_size": 512,
    "epochs": 100,
    "lr": 0.001,
    "momentum": 0.9,
}

dataloaders = init_dataloaders(batch_size)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                                              T_max=10,  # Maximum number of iterations
                                                                              eta_min=0,  # 最小学習率
                                                                              last_epoch=-1)  # The index of last epoch

model = train(model, criterion, optimizer, scheduler, params)

In [ ]:
calc_acc(model)

StepLR

In [ ]:
params = {
    "batch_size": 1024,
    "epochs": 50,
    "lr": 0.001,
    "momentum": 0.9,
}

dataloaders = init_dataloaders(batch_size)
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.StepLR(optimizer=optimizer,
                                                     step_size=10,
                                                     gamma=0.9)

model = train(model, criterion, optimizer, scheduler, params)

In [ ]:
calc_acc(model)

# Transfer Learning

## Finetuning ResNet-18

In [ ]:
data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

In [ ]:
trainset = torchvision.datasets.CIFAR10(root="../../data",
                                                                       train=True,
                                                                       download=True,
                                                                       transform=transforms.Compose([
                                                                           transforms.RandomResizedCrop(224),
                                                                           transforms.RandomHorizontalFlip(),
                                                                           transforms.ToTensor(),
                                                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                                                       ]))
dataloader[TRAIN] = torch.utils.data.DataLoader(trainset,
                                                                           batch_size=batch_size,
                                                                           shuffle=True,
                                                                           num_workers=2)

testset = torchvision.datasets.CIFAR10(root="../../data",
                                                                      train=False,
                                                                      download=True,
                                                                       transform=transforms.Compose([
                                                                           transforms.Resize(256),
                                                                           transforms.RandomResizedCrop(224),
                                                                           transforms.ToTensor(),
                                                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                                                       ]))
test_loader = torch.utils.data.DataLoader(testset,
                                                                          batch_size=test_batch_size,
                                                                          shuffle=False)

In [ ]:
params = {
    "batch_size": 16,
    "epochs": 10,
    "lr": 0.001,
    "momentum": 0.9,
}

data_transforms = {
    "train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ]),
    "val": transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
}

image_datasets, dataset_sizes = init_datasets(data_transforms)
dataloaders = init_dataloaders(params["batch_size"], image_datasets)

# Load pretrained model
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(in_channels=3,
                                                    out_channels=64,
                                                    kernel_size=7,
                                                    stride=2,
                                                    padding=3,
                                                    bias=False)
model_ft.fc = nn.Linear(num_ftrs, 10)
model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_ft.parameters(),
                                            lr=params["lr"])
scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,
                                                                              T_max=10,  # Maximum number of iterations
                                                                              eta_min=0,  # 最小学習率
                                                                              last_epoch=-1)  # The index of last epoch

model_ft = train(model_ft, criterion, optimizer, scheduler, params)

In [ ]:
since = time.time()

correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))

display_formatted_time(time.time() - since)

### Fixed feature extractor

In [ ]:
model_ft = models.resnet18(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.fc.in_features
model_ft.conv1 = nn.Conv2d(in_channels=3,
                                                    out_channels=64,
                                                    kernel_size=7,
                                                    stride=2,
                                                    padding=3,
                                                    bias=False)
model_ft.fc = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)
print(model_ft)

optimizer_ft = optim.SGD(model_ft.parameters(),
                                              lr=lr,
                                              momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
since = time.time()

for epoch in range(epochs):

    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(dataloader[TRAIN], 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        exp_lr_scheduler.step()

        running_loss += loss.item()
        if batch_idx % log_interval == (log_interval - 1):
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tRunningLoss: {:.3f}".format(
                epoch, batch_idx * len(inputs), len(dataloader[TRAIN].dataset),
                100. * batch_idx / len(dataloader[TRAIN]), loss.item(), running_loss / log_interval
            ))
            running_loss = 0.0

display_formatted_time(time.time() - since)

In [ ]:
since = time.time()

correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))

display_formatted_time(time.time() - since)

## Finetuning VGG

In [ ]:
model_ft = models.vgg16(pretrained=True)
for param in model_ft.parameters():
    param.requires_grad = False

num_ftrs = model_ft.classifier[6].in_features

model_ft.classifier[6] = nn.Linear(num_ftrs, 10)

model_ft = model_ft.to(device)
print(model_ft)

optimizer_ft = optim.SGD(model_ft.parameters(),
                                              lr=lr,
                                              momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
since = time.time()

for epoch in range(epochs):

    running_loss = 0.0
    for batch_idx, (inputs, labels) in enumerate(dataloader[TRAIN], 0):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model_ft(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        exp_lr_scheduler.step()

        running_loss += loss.item()
        if batch_idx % log_interval == (log_interval - 1):
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tRunningLoss: {:.3f}".format(
                epoch, batch_idx * len(inputs), len(dataloader[TRAIN].dataset),
                100. * batch_idx / len(dataloader[TRAIN]), loss.item(), running_loss / log_interval
            ))
            running_loss = 0.0

display_formatted_time(time.time() - since)

In [ ]:
since = time.time()

correct = 0
total = 0
with torch.no_grad():
    for (inputs, labels) in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_ft(inputs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print("Accuracy of the network on the 10000 test images: %d %%" % (100 * correct / total))

display_formatted_time(time.time() - since)

In [ ]:
print(labels)
print(outputs)